Unused code for number of unique users. 

Next, the number of unique accounts posting about the topic each day is counted. On days where there was a surge of tweets, the data only returns tweets for up to a certain time instead of the full day. Due to these limitations, the tweet data will be combined with the number of tweets dataset to estimate the number of unique accounts posting on that day. 

Read the tweet data to dataframes. Like for the wordclouds, only the data with tweets from accounts with over 20 followers is kept to get rid of spam.
Using the 'df_number_of_tweets_by_day' dataframe, add a column with the time of last tweet in tweet dataset and number of unique users. The dataset is sorted from descending time.

Only the data for 20 and 27 November is complete. So the number of seconds that we have data for will be calculated. Then, the number of tweets per second and number of unique users per second is calcuated to get the number of unique users per tweet.

Since the data for tweets is by day, the data for news articles will be split up by day as well.

In [ ]:
df_number_of_tweets["Date"] = df_number_of_tweets["Date"].astype(str)
df_number_of_tweets[['Day','Time']] = df_number_of_tweets.Date.str.split(expand=True)

df_number_of_tweets.head()


df_tweets_by_day = df_number_of_tweets.groupby("Day")["Number of tweets"].sum().reset_index(name='Number of tweets')

df_tweets_by_day.head(8)

dataframe = {}

for date in range(20, 28):
    file = "data/" + str(date) +"_nov_archive.csv"
    df = pd.read_csv(file)
    key = str("df_" + str(date) + "_nov")
    dataframe[key] = df

for date in dataframe:
    dataframe[date] = dataframe[date][dataframe[date]["user_followers_count"] > 49] 

time_of_last_tweet = []
number_of_unique_users_in_dataset = []

for date in dataframe:
    df = dataframe[date]
    df[["Day","Time"]] = df.time.str.split(expand=True)
    time_of_last_tweet.append(df.iloc[-1]["Time"])
    number_of_unique_users_in_dataset.append(df["from_user"].unique().size)

df_tweets_by_day["Time of last tweet"] = time_of_last_tweet
df_tweets_by_day["Number of unique users"] = number_of_unique_users_in_dataset


df_tweets_by_day["Time of last tweet"]= pd.to_timedelta(df_tweets_by_day["Time of last tweet"])
df_tweets_by_day["Seconds"] = df_tweets_by_day["Time of last tweet"].dt.total_seconds()

df_tweets_by_day["Tweets per second"] = df_tweets_by_day["Number of tweets"]/(24*60*60)
df_tweets_by_day["Unique users per second"] = df_tweets_by_day["Number of unique users"]/((24*60*60) - df_tweets_by_day["Seconds"])
df_tweets_by_day["Total unique users"] = (df_tweets_by_day["Unique users per second"]/df_tweets_by_day["Tweets per second"])*(df_tweets_by_day["Number of tweets"])

df_tweets_by_day.loc[0, "Total unique users"] = df_tweets_by_day.loc[0, "Number of unique users"]
df_tweets_by_day.loc[7, "Total unique users"] = df_tweets_by_day.loc[7, "Number of unique users"]

df_tweets_by_day["Total unique users"] = df_tweets_by_day["Total unique users"].round(0).astype(int)

df_news_articles_stats_by_day = df_news_articles_stats
df_news_articles_stats_by_day["Date"] = df_news_articles_stats_by_day["Date"].astype(str)
df_news_articles_stats_by_day[["Day","Time"]] = df_news_articles_stats.Date.str.split(expand=True)

trace1 = go.Scatter(
    x = df_news_articles_stats_by_day["Day"], 
    y = df_news_articles_stats_by_day["y"] + 100000,
    name = "Reach of news articles and posts",
    mode = 'markers',
    text = df_news_articles_stats_by_day["News agency"],
    marker = dict(
        color = df_news_articles_stats_by_day["Color"],
        size = df_news_articles_stats_by_day["Total number of engagements"],
        sizemode = 'area',
        sizeref = 2.*max(df_news_articles_stats_by_day["Total number of engagements"])/(40.**2),
        sizemin = 4
    ),
    hovertemplate = "Date posted: %{x}  <br>News agency: %{text} <br>Reach: %{marker.size} <extra></extra>"
)


trace2 = go.Scatter(
    x = df_tweets_by_day["Day"], 
    y = df_tweets_by_day["Total unique users"],
    marker_color="rgba(152, 0, 0, .8)",
    name = "Total unique users"
)

fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    hoverlabel_align = 'right')
fig.update_layout(hovermode="x unified")
fig.show()